## 모델 훈련 및 평가

### 내용
1. 모델 훈련 기본 개념
    - 모델 훈련 과정
        - 순전파(forward pass): 입력 데이터를 모델에 통과시켜 예측값 계산
        - 손실 계산: 예측값과 실제값의 차이 계산
        - 역전파(backward pass): 기울기 계산
        - 가중치 업데이트: 옵티마이저를 통해 가중치 조정
    - 손실 함수(Loss Function)
        - 손실 함수는 모델의 예측값과 실제값의 차이를 측정하는 함수
        - nn.MSELoss(), nn.CrossEntropyLoss()

In [1]:
import torch
import torch.nn as nn

In [2]:
# MSELoss 예시
criterion = nn.MSELoss()

- 옵티마이저(Optimizer)
    - 옵티마이저는 손실을 줄이기 위해 가중치를 업데이트하는 알고리즘
    - optim.SGD, optim.Adam

In [4]:
import torch.optim as optim

In [5]:
model = nn.Sequential(
    nn.Linear(10, 50),
    nn.ReLU(),
    nn.Linear(50, 1)
)

# SGD 옵티마이저 예시
optimizer = optim.SGD(model.parameters(), lr=0.01)

2. 모델 훈련 루프
    - 훈련 루프 작성

In [8]:
# 1. 모델 정의 (이전 주차의 예제 사용)
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()

In [9]:
# 2. 손실 함수와 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [11]:
# 3. 데이터 로더 설정
from torch.utils.data import DataLoader, TensorDataset

data = torch.randn(100, 10)
labels = torch.randn(100, 1)
dataset = TensorDataset(data, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [12]:
# 4. 훈련 루프
num_epochs = 100

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        optimizer.zero_grad()  # 옵티마이저 초기화
        outputs = model(inputs)  # 순전파
        loss = criterion(outputs, targets)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/100, Loss: 0.9002809524536133
Epoch 2/100, Loss: 1.0825366973876953
Epoch 3/100, Loss: 1.0203309059143066
Epoch 4/100, Loss: 1.0465378761291504
Epoch 5/100, Loss: 2.878781795501709
Epoch 6/100, Loss: 1.5387649536132812
Epoch 7/100, Loss: 0.7956857681274414
Epoch 8/100, Loss: 2.389129638671875
Epoch 9/100, Loss: 0.37842002511024475
Epoch 10/100, Loss: 0.1155659630894661
Epoch 11/100, Loss: 0.26015201210975647
Epoch 12/100, Loss: 0.5031731128692627
Epoch 13/100, Loss: 1.7087327241897583
Epoch 14/100, Loss: 0.7222334146499634
Epoch 15/100, Loss: 0.6265220046043396
Epoch 16/100, Loss: 1.585899829864502
Epoch 17/100, Loss: 0.7409024238586426
Epoch 18/100, Loss: 0.46957916021347046
Epoch 19/100, Loss: 0.700609028339386
Epoch 20/100, Loss: 0.011972482316195965
Epoch 21/100, Loss: 1.1783714294433594
Epoch 22/100, Loss: 0.3350522816181183
Epoch 23/100, Loss: 0.7000587582588196
Epoch 24/100, Loss: 0.5525322556495667
Epoch 25/100, Loss: 0.940446138381958
Epoch 26/100, Loss: 1.084882855415

3. 모델 평가    
    - 검증 데이터셋을 사용한 평가
        - 훈련 중간 및 끝에 모델 성능을 평가하기 위해 검증 데이터셋 사용
        - 모델의 일반화 성능 평가

In [13]:
# 검증 데이터 로더 설정
val_data = torch.randn(20, 10)
val_labels = torch.randn(20, 1)
val_dataset = TensorDataset(val_data, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [14]:
# 모델 평가 함수
def evaluate_model(model, val_loader):
    model.eval()  # 평가 모드
    val_loss = 0
    with torch.no_grad():  # 평가 시 기울기 계산 안함
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    return val_loss / len(val_loader)

In [15]:
# 훈련 후 평가
val_loss = evaluate_model(model, val_loader)
print(f'Validation Loss: {val_loss}')

Validation Loss: 0.9518161714076996


- 모델 성능 지표
    - 정확도(Accuracy), 정밀도(Precision), 재현율(Recall), F1-score 등
    - 회귀 문제에서는 MSE, RMSE, MAE 등

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
# 예시: 분류 문제에서 정확도 계산
def calculate_accuracy(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

4. 실습 및 과제
    - 주어진 데이터셋을 사용하여 신경망 모델을 구축하고, 훈련 및 평가
    - 손실 함수와 옵티마이저를 설정하고, 훈련 루프 작성
    - 검증 데이터셋을 사용하여 모델 성능 평가

In [18]:
# 실습 과제 예시
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error


In [19]:
# 데이터 생성
data = torch.randn(100, 10)  # 100개의 샘플, 각 샘플은 10개의 특징
labels = torch.randn(100, 1)  # 100개의 샘플에 대한 타겟 값

# 데이터셋 및 데이터로더 설정
dataset = TensorDataset(data, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# 모델 정의
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()

# 손실 함수와 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [20]:
# 훈련 루프
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # 훈련 모드
    for inputs, targets in dataloader:
        optimizer.zero_grad()  # 옵티마이저 초기화
        outputs = model(inputs)  # 순전파
        loss = criterion(outputs, targets)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


Epoch 1/100, Loss: 1.2030823230743408
Epoch 2/100, Loss: 0.2133454978466034
Epoch 3/100, Loss: 0.20094585418701172
Epoch 4/100, Loss: 1.8279443979263306
Epoch 5/100, Loss: 1.0112817287445068
Epoch 6/100, Loss: 0.18924736976623535
Epoch 7/100, Loss: 0.16863860189914703
Epoch 8/100, Loss: 0.10916955024003983
Epoch 9/100, Loss: 0.15671765804290771
Epoch 10/100, Loss: 0.8185521960258484
Epoch 11/100, Loss: 1.2489330768585205
Epoch 12/100, Loss: 0.6655643582344055
Epoch 13/100, Loss: 2.2120189666748047
Epoch 14/100, Loss: 0.4364643096923828
Epoch 15/100, Loss: 1.1606277227401733
Epoch 16/100, Loss: 0.9663739800453186
Epoch 17/100, Loss: 0.575304388999939
Epoch 18/100, Loss: 1.118754506111145
Epoch 19/100, Loss: 0.24422818422317505
Epoch 20/100, Loss: 2.430311679840088
Epoch 21/100, Loss: 1.1744812726974487
Epoch 22/100, Loss: 1.0729669332504272
Epoch 23/100, Loss: 0.3275945782661438
Epoch 24/100, Loss: 0.5875079035758972
Epoch 25/100, Loss: 1.0371150970458984
Epoch 26/100, Loss: 1.295012354

In [21]:
# 모델 평가
val_data = torch.randn(20, 10)
val_labels = torch.randn(20, 1)
val_dataset = TensorDataset(val_data, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

def evaluate_model(model, val_loader):
    model.eval()  # 평가 모드
    val_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            all_preds.extend(outputs.tolist())
            all_labels.extend(labels.tolist())

    val_loss /= len(val_loader)
    mse = mean_squared_error(all_labels, all_preds)
    return val_loss, mse

val_loss, mse = evaluate_model(model, val_loader)
print(f'Validation Loss: {val_loss}, MSE: {mse}')

Validation Loss: 1.6823636293411255, MSE: 1.3099048036617345
